In this notebook, we explore basic analysis of the positions and speeds to detect similarities and differences between behaviouirs.

We start importing some libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

import perdiver.perdiver as perdiver

from navground import core, sim

plots_dir = os.path.join("plots", "visualisation")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

In [ ]:
# Cross
side = 2
num_steps = 400

num_agents = 11 # 31
num_runs = 1
behaviour_list = ["ORCA", "HL", "HRVO", "Dummy"]
marker_behaviour = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*"}
color_behaviour = {}
for i, behaviour in enumerate(behaviour_list):
    color_behaviour[behaviour] = mpl.colormaps["Set1"](i / (len(behaviour_list) +1))
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"visualisation_{behaviour}.h5")
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: {num_runs}
    record_collisions: true
    record_deadlocks: true
    record_safety_violation: true
    record_efficacy: true
    terminated_when_idle_or_stuck: false
    scenario:
      type: Cross
      side: {side} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 
                sampler: normal
                mean: 0.2
                std_dev: 0.05
            horizon: 5.0
            safety_margin: 
                sampler: normal
                mean: 0.2
                std_dev: 0.05
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

In [ ]:
runs = {}
# Reload simulations
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"visualisation_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

Now, we illustrate movements for each behaviour in a single image.

In [ ]:
def plot_image_cross(run, image, timestep_list, side, ax):
    # Load poses and number of agents
    poses = run.poses
    num_agents = poses.shape[1]
    # Plot image and adjust bounds
    ax.imshow(image, extent=[-side/2, side/2, -side/2, side/2])
    ax.set_xlim([-side/2, side/2])
    ax.set_ylim([-side/2, side/2])
    # Plot trajectories
    for i in range(num_agents):
        for f, t in zip(timestep_list[:-1], timestep_list[1:]):
            edge = np.array([poses[f,i], poses[t,i]])
            ax.plot(edge[:,0], edge[:,1], color=mpl.colormaps['Set1'](i/(num_agents+3)))
        # end for
    # end for

In [ ]:
fig, ax = plt.subplots(ncols=len(behaviour_list), figsize=(4*len(behaviour_list),4))
timestep_list = list(range(60,140,15))
for i, behaviour in enumerate(behaviour_list):
    run = runs[behaviour][0]
    image = perdiver.image_run_timestep_list(run, timestep_list)
    plot_image_cross(run, image, timestep_list, side, ax[i])
    ax[i].set_title(behaviour, fontsize=20)

plt.savefig(os.path.join(plots_dir, "behaviours_cross.png"))